In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


def calculate_point_estimate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    nrmse = rmse / (np.max(y_true) - np.min(y_true))
    wape = np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100
    wmape = np.average(np.abs(y_true - y_pred) / np.abs(y_true), weights=np.abs(y_true)) * 100

    return {
        "R-Squared (R²)": r2,
        "Mean Absolute Error (MAE)": mae,
        "Mean Absolute Percentage Error (MAPE)": mape,
        "Mean Squared Error (MSE)": mse,
        "Root Mean Squared Error (RMSE)": rmse,
        "Normalized RMSE": nrmse,
        "WAPE": wape,
        "WMAPE": wmape
    }


def calculate_interval_metrics(y_true, lower, upper, alpha=0.1):
    coverage = np.mean((y_true >= lower) & (y_true <= upper))
    width = np.mean(upper - lower)
    score = (upper - lower) + \
            (2 / alpha) * (lower - y_true) * (y_true < lower) + \
            (2 / alpha) * (y_true - upper) * (y_true > upper)
    interval_score = np.mean(score)

    return {
        "Coverage Rate": coverage,
        "Average Interval Width": width,
        "Interval Score": interval_score
    }


def evaluate_and_save_metrics(csv_path, lower_col, upper_col, median_col, true_col, alpha, output_csv_path):
    df = pd.read_csv(csv_path)

    y_true = df[true_col].values
    y_pred = df[median_col].values
    lower = df[lower_col].values
    upper = df[upper_col].values

    point_metrics = calculate_point_estimate_metrics(y_true, y_pred)
    interval_metrics = calculate_interval_metrics(y_true, lower, upper, alpha)

    # Combine all metrics into a DataFrame
    all_metrics = {**point_metrics, **interval_metrics}
    metrics_df = pd.DataFrame(list(all_metrics.items()), columns=["Metric", "Value"])

    # Save metrics to CSV
    metrics_df.to_csv(output_csv_path, index=False)
    print(f"Saved evaluation metrics to {output_csv_path}")


# === CONFIGURATION ===
asset = "Bond"
model_type = "TFT"  # or "LSTM"
target_type = "Return"  # or "Volatility"

# Input file: should contain "Actual", "Predicted_5%", "Predicted_50%", "Predicted_95%" columns
input_csv = r""

# Output file for metrics
output_metrics_csv = r""

# Run evaluation
print(f"{model_type} {asset} {target_type} - Evaluation")
evaluate_and_save_metrics(
    csv_path=input_csv,
    lower_col="Predicted_5%",
    upper_col="Predicted_95%",
    median_col="Predicted_50%",
    true_col="Actual",
    alpha=0.1,
    output_csv_path=output_metrics_csv
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410 entries, 0 to 409
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  410 non-null    int64  
 1   Date                        410 non-null    object 
 2   CPI_actual                  408 non-null    float64
 3   Unemployemnt Rate           410 non-null    float64
 4   1-Year T-Bill Rate          410 non-null    float64
 5   10-Year T-Bill Rate         410 non-null    float64
 6   PMI_Actual                  409 non-null    float64
 7   CPI_Forecast_of_this_month  408 non-null    float64
 8   Bond_Return_this_month      410 non-null    float64
 9   Bond_Vol_this_month         410 non-null    float64
 10  CPI_Forecast_next_month     409 non-null    float64
 11  Target_Return               410 non-null    float64
 12  Target_Vol                  410 non-null    float64
 13  Target_Month                410 non